### import
- 기준일 다음날 정식,약식데이터(목록을 최대한 비슷하게 하기 위해 정식도 기준일 다음날 받는다.)
- 컷오프요일이 언제든 금액정보는 정확히 반영할 수 있음

In [2]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

### 파일 읽기

In [4]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
약식병합 = False # 금요일(일요일)이 컷오프가 아니라면 약식병합해야-------
basedate = "240531"
company = "대성"  # 통합 or 대성, 통합=솔림+신탁88
########################################################

# 읽을 파일명
filename_read = basedate + "_계좌별 진행상황 조회_" + company

# 파일 읽기
if 약식병합 : 
    suffix_정식 = "_업로드결과.xlsx"
    suffix_약식 = "_약식_업로드결과.xlsx"

    # 계좌별 진행상황 조회 파일 읽기
    정식_ori = pd.read_excel(join(path_base, filename_read + suffix_정식 ), dtype=nauri_dtype).fillna("")
    약식_ori = pd.read_excel(join(path_base, filename_read + suffix_약식 ), dtype=nauri_dtype).fillna("")
else : 
    suffix_정식 = "_업로드결과.xlsx"
    정식_ori = pd.read_excel(join(path_base, filename_read + suffix_정식 ), dtype=nauri_dtype).fillna("")
    
# 계좌 (메모 : 통합신고건 처리위해) --------------
if company == "통합" : 
    dir_account = join(r"D:\3.자산\전산 dataset", "솔림", basedate)
else : 
    dir_account = join(r"D:\3.자산\전산 dataset", "대성", basedate)
file_list = os.listdir(dir_account)
account_filename = [file for file in file_list if file.startswith("계좌조회새창")][0]
account = pd.read_excel(join(dir_account, account_filename), dtype=account_dtype)[["계좌키","채무자키","메모"]].fillna("")

### 전처리

##### 키 중복건 처리

In [5]:
# 재시작 위해 복사본 만들기
if 약식병합 : 
    약식 = 약식_ori.copy()
    print("약식건수", len(약식))
    # 키값에 중복건 있는지 확인, merge때문에 약식, 정식 모두 해야함
    약식["pk"] = 약식.신청인명 + 약식.신청인주민번호 + 약식.접수번호 + 약식.계좌번호
    print("약식 키 중복건")
    display(약식[약식.pk.duplicated(keep=False)])

정식 = 정식_ori.copy()
print("정식건수", len(정식))

정식["pk"] = 정식.신청인명 + 정식.신청인주민번호 + 정식.접수번호 + 정식.계좌번호
print("정식 키 중복건")
display(정식[정식.pk.duplicated(keep=False)])

정식건수 979
정식 키 중복건


,채무자키,계좌키,보증인키,신용회복키,채무자명,채무상태,현재채권구분,신청인명,신청인주민번호,접수번호,...,등록파일,수정파일,삭제여부,등록일자,등록유저,수정일자,수정유저,확인결과,저장결과,pk


In [6]:
# 중복건 확인 후 삭제 : 솔림, 신탁 중복 신고건일 것이다.
####################################################################
정식 = 정식.drop_duplicates(subset=["pk"], keep='first', inplace=False, ignore_index=True)

In [7]:
if 약식병합 : 
        # 정식 : 불필요 칼럼 삭제
        키열 = ["신청인명", "신청인주민번호", "접수번호", "계좌번호"]

        나우리칼럼불요 = ["채무자명", "채무상태", "현재채권구분","신청인진행상태변경전", "특별상환유예", "pk", # pk 추가
                "변경일", "변경구분", "신용회복진행상태","신복아이디","등록파일","수정파일","삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"]
        나우리칼럼필요 = ["채무자키", "계좌키", "보증인키", "신용회복키"]
        약식열 = ["거래일","심의차수","신청인진행상태","계좌진행상태내용","실효/완제/합의서포기일자","접수통지일","접수일","점포명","확정일","합의서체결일","상환개시일",
                "최초대출원금","조정전원금","조정전이자","조정전연체이자","조정전비용","조정전합계","조정후원금","조정후이자","조정후연체이자","조정후비용","조정후합계",
                "원금감면여부","조정대상포함여부","총상환기간","유예기간","원금균등상환기간","원리균등상환기간","이자상환기간","납입회차","연체기간","총납입금액","총납입원금",
                "총납입이자","총납입기타채무","상환후잔액","감면율","급여가압류원금","채무구분","주채무자주민번호","주채무자성명","동의요청일","동의확정일","재조정여부","재조정횟수",
                "수정조정여부","수정조정횟수","재조정접수일자","재조정처리일자","재조정반송일자","실효취소요청일자","실효원상회복일자","상각구분","신청구분","일시납감면율",
                "차상위계층여부","미납발생일","부동산담보대출여부","거치기간","감면방식","상환방식","담보권실행유예기간","특별면책","재난상환유예","특례지원","이자율재조정","거치이자납입유예"]

        정식.drop(columns= 나우리칼럼불요 + 나우리칼럼필요 + 약식열, inplace=True, axis=1)

        # 약식 : 신복 약식에 있는 칼럼만 남기기
        약식 = 약식.loc[:, 나우리칼럼필요 + 키열 + 약식열]
        
        # merge : 약식을 기준으로 정식에만 있는 칼럼 merge(약식에도 있는 칼럼은 drop했으므로 그대로 붙이면 됨)
        raw = pd.merge(약식, 정식, on=["신청인명", "신청인주민번호", "접수번호", "계좌번호"], how='left')
        print(len(raw), "병합전 약식 개수 : ", len(약식))
        
else : 
        raw = 정식

In [8]:
# 신용회복 자료와 칼럼 순서 맞추기 : 재난상환유예를 특별면책 뒤, 특례지원 앞으로
cols = ["채무자키","계좌키","보증인키","신용회복키","신청인명","신청인주민번호","접수번호","거래일","계좌번호","심의차수","신청인진행상태","계좌진행상태내용","실효/완제/합의서포기일자",
        "접수통지일","접수일","점포명","확정일","합의서체결일","상환개시일","대출과목","조정전이율","조정후이율","최초대출원금","조정전원금","조정전이자","조정전연체이자","조정전비용",
        "조정전합계","조정후원금","조정후이자","조정후연체이자","조정후비용","조정후합계","원금감면여부","조정대상포함여부","총상환기간","유예기간","원금균등상환기간","원리균등상환기간",
        "이자상환기간","납입회차","연체기간","총납입금액","원금균등시작회차","원금균등종료회차","원리균등시작회차","원리균등종료회차","이자상환시작회차","이자상환종료회차","원금균등채무액",
        "총납입원금","총납입이자","총납입기타채무","상환후잔액","감면율","추심원금","급여가압류원금","채무구분","주채무자주민번호","주채무자성명","시작회차1","종료회차1","채무조정1적용이율1",
        "채무조정2적용이율1","채무조정1월상환액1","채무조정2월상환액1","시작회차2","종료회차2","채무조정1적용이율2","채무조정2적용이율2","채무조정1월상환액2","채무조정2월상환액2","시작회차3",
        "종료회차3","채무조정1적용이율3","채무조정2적용이율3","채무조정1월상환액3","채무조정2월상환액3","시작회차4","종료회차4","채무조정1적용이율4","채무조정2적용이율4","채무조정1월상환액4",
        "채무조정2월상환액4","시작회차5","종료회차5","채무조정1적용이율5","채무조정2적용이율5","채무조정1월상환액5","채무조정2월상환액5","시작회차6","종료회차6","채무조정1적용이율6","채무조정2적용이율6",
        "채무조정1월상환액6","채무조정2월상환액6","시작회차7","종료회차7","채무조정1적용이율7","채무조정2적용이율7","채무조정1월상환액7","채무조정2월상환액7","시작회차8","종료회차8","채무조정1적용이율8",
        "채무조정2적용이율8","채무조정1월상환액8","채무조정2월상환액8","시작회차9","종료회차9","채무조정1적용이율9","채무조정2적용이율9","채무조정1월상환액9","채무조정2월상환액9","시작회차10","종료회차10",
        "채무조정1적용이율10","채무조정2적용이율10","채무조정1월상환액10","채무조정2월상환액10","동의요청일","동의확정일","재조정여부","재조정횟수","수정조정여부","수정조정횟수","재조정이전납입금액",
        "원금완제면제이자","출금정지등록여부","재조정접수일자","재조정처리일자","재조정반송일자","실효취소요청일자","실효원상회복일자","상각구분","신청구분","일시납감면율","차상위계층여부","미납발생일",
        "부동산담보대출여부","거치기간","감면방식","상환방식","체증식계좌여부","체증전구간의시작회차","체증전구간의종료회차","체증전구간의기준기간","변제금수취계좌","담보권실행유예기간","특별면책",
        "재난상환유예","특례지원","이자율재조정","거치이자납입유예"
]
raw = raw.loc[:,cols]


### 반영OPB 등

In [9]:
# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 4
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

# 전산미등록건 처리 #####################################################
if company == "통합" : 
    미등록주민번호 = ['771027-2221231'] #강민경
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
else :  # 대성
    미등록주민번호 = ['741008-1037838'] #강승욱
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
########################################################################

In [10]:
# 주민 계좌 중복건 : 전산에는 중복건 있더라도 신복위에서 받은 데이터에는 없는것이 당연(솔림,신탁 동시 채권신고요청시에는 있을 수 있음)
raw[raw.duplicated(subset=["신청인주민번호", "계좌번호"], keep=False)]

,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,접수번호,...,체증전구간의시작회차,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,재난상환유예,특례지원,이자율재조정,거치이자납입유예


In [11]:
# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
]
values = ["확정", "채무조정제외","완제","채권신고전","실효"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위  
# 정상건인데 상환후잔액이 없는 경우는, 실효취소 및 재조정합의 후 첫회차 입금이 안 된 경우
# 조정후비용이나 이자가 있는 경우가 있기 때문에 원래 2순위로 조정후 합계가 반영되어야 하나, 회계법인 평가법에 아래와 같이 나와 있고, 이를 다른 회사도 적용하고 있다고 해서 그냥 이대로 적용하기로.
# 또한 약식을 활용하면 상환후잔액이 없는 경우가 거의 없어서 큰 문제는 없음.
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금 + raw.조정전비용   # 조정정비용 추가 2024-04-03

# 미확정-조정전원금0 은 이자채권으로 그냥 0이 맞다.
conds = [(x == "미확정"), (x =="확정")&(a!=0), b!=0, c!=0, d!=0]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = np.nan

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.상환방식

conds = [x.str.contains("원금균등"), x.str.contains("원리금균등")] #체증식(원금균등)이란게 있어서 contains로 바꿈

# 조건에 따라 값을 설정
values = ["개인","프리"]

# 기본값 설정
default_value = "" # 미확정

# 반영OPB 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

### 시트작성 및 체크 : 최종,정상,제외

In [12]:
# 최종, 확정미확정, 일부계좌제외 시트
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
확정미확정 = raw.query('진행구분 in ["확정", "미확정"]').copy()
일부계좌제외 = raw.query('진행구분 == "채무조정제외" and 채무자키 != ""') # 계좌번호 변경한 건들 있어서 채무자키 없을 수 있음, 20240117 기준 솔림 8건
# 일부계좌제외는 자산정리 채무상태에서 신복으로 처리하여 오류탐지 제외 ~~

# 전산미등록건, 일부계좌제외
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)
확정미확정.drop(확정미확정[확정미확정.채무자키=='전산미등록건'].index, inplace=True)
일부계좌제외.drop(일부계좌제외[일부계좌제외.채무자키=='전산미등록건'].index, inplace=True)

# 포함과 제외가 공존하는 제외건만 : 제외자 주민번호가 확정미확정에도 있으면
일부계좌제외 = 일부계좌제외.merge(확정미확정["신청인주민번호"], how="inner", on=["신청인주민번호"])

# 순번열 넣기
if "순번" not in raw.columns : 
    raw.insert(0, "순번", range(1,len(raw)+1))
    최종.insert(0, "순번", range(1,len(최종)+1))
    확정미확정.insert(0, "순번", range(1,len(확정미확정)+1))
    일부계좌제외.insert(0, "순번", range(1,len(일부계좌제외)+1))

In [13]:
# 체크
# 반영OPB
반영OPB오류 = raw[(raw.반영OPB == np.nan)|(raw.반영OPB == 0)]
cols = ["채무자키", "계좌키", "진행구분", "상환구분","반영OPB", "조정전원금"]
if len(반영OPB오류) > 0 : 
    print('반영OPB 0원 ------')
    print('원금,법비용 0원계좌라도 상환예정액 조금이라도 나옴. 미확정건이면 전산opb를 따르니 0원 되겠지') ######################
    display(반영OPB오류[cols])
else : print("반영OPB 오류 없음")

# 확정미확정에 신용회복키 중복여부
신용회복키중복 = 확정미확정[확정미확정.duplicated('신용회복키', keep=False)]
if len(신용회복키중복) > 0 : 
    print('신용회복키 중복건 ------ 신용회복키 빈값 없애려면 업로드 후에 재다운로드해야')
    display(신용회복키중복)
else : print("확정미확정에 신용회복키 중복 없음")

# 채무자키, 계좌키, 신용회복키 매칭 오류
최종키오류 = 최종.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
확정미확정키오류 = 확정미확정.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
print(f"전체 : {len(raw)}, 최종 : {len(최종)}, 확정미확정 : {len(확정미확정)}")
print(f"최종에서 키 없는 행 수 : {len(최종키오류)}, 확정미확정에서 키없는 행 수 : {len(확정미확정키오류)}")
if len(확정미확정키오류)>0 : 
    print('확정미확정 채무자키, 계좌키, 신복키 중 하나라도 없는 경우 ------')
    display(확정미확정키오류)
else : print("확정미확정 건에 키매칭 실패 없음")

# 확정 건 중 상환구분 없는 값
print("확정 건 중 상환구분 없는 값 :", len(확정미확정.query('상환방식=="확정" and 상환구분==""')))

# 일부계좌제외에 보증채무있는지
print("일부계좌제외에 보증채무수 :", len(일부계좌제외.query('채무구분=="보증채무"'))) # 있으면 드랍할지 말지 그때 결정

반영OPB 오류 없음
확정미확정에 신용회복키 중복 없음
전체 : 979, 최종 : 950, 확정미확정 : 661
최종에서 키 없는 행 수 : 9, 확정미확정에서 키없는 행 수 : 0
확정미확정 건에 키매칭 실패 없음
확정 건 중 상환구분 없는 값 : 0
일부계좌제외에 보증채무수 : 0


### 통합신고건 처리하기

In [14]:
통합신고계좌 = account.query("메모.str.contains('신용회복 통합신고')")
print("통합신고계좌수 :",len(통합신고계좌))
print("추가전 확정미확정 건수 :", len(확정미확정))

cnt = 확정미확정.순번.iloc[-1]

for i, v in 통합신고계좌.iterrows() : 
    피통합계좌 = re.search(r"[0-9]{9,9}", v.메모)[0]
    temp = 확정미확정.query('계좌키 == @피통합계좌').copy()
    if len(temp) == 1 :
        cnt += 1
        temp['순번'] = cnt
        temp['채무자키'] = v.채무자키
        temp['계좌키'] = v.계좌키
        temp[['신용회복키','계좌진행상태내용']] = '통합신고 ' + v.계좌키
        temp['반영OPB'] = 0
        
        # 행추가
        확정미확정 = pd.concat([확정미확정, temp])
    elif len(temp) == 0 :
        print("통합신고 취소, 원상회복 분할적용 확인 : ", v.계좌키, v.채무자키)
    
    else : 
        print("중복계좌키 발견 : ", v.계좌키, v.채무자키)
        
print("추가후 확정미확정 건수 :", len(확정미확정))

전체요약 = 확정미확정.groupby('진행구분').agg({'신용회복키':'count', '반영OPB':'sum'})
전체요약.loc["합계", :] = 전체요약.sum()
display(전체요약.applymap('{:,.0f}'.format))

통합신고계좌수 : 0
추가전 확정미확정 건수 : 661
추가후 확정미확정 건수 : 661


,신용회복키,반영OPB
진행구분,,
미확정,8,"27,470,666"
확정,653,"711,069,527"
합계,661,"738,540,193"


### 파일 저장

In [15]:
# 저장
# 기준데이터 저장 경로
if 약식병합 : 
    filename_write = basedate+"_신용회복_기준데이터_약식병합_"+ ("솔림" if company=="통합" else "대성")
else : 
    filename_write = basedate+"_신용회복_기준데이터_"+ ("솔림" if company=="통합" else "대성")
    
fullpath_result = join(os.path.dirname(path_base),"기준데이터", filename_write +".xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    raw.to_excel(writer, sheet_name='원본', index=False)
    최종.to_excel(writer, sheet_name='최종', index=False)
    확정미확정.to_excel(writer, sheet_name='확정,미확정', index=False)
    일부계좌제외.to_excel(writer, sheet_name='일부계좌제외', index=False)